In [1]:
import pandas as pd
import os
from rapidfuzz import process, fuzz
from tqdm import tqdm
tqdm.pandas()

/Users/Lara/anaconda3/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data_dir = "/Users/Lara/Documents/Stanford/Research/Glassdoor"

In [3]:
glassdoor_reviews = pd.read_csv(os.path.join(data_dir, 'reviews_new_processed.csv'), usecols=['employer_id', 'employer_name'])
glassdoor_reviews = glassdoor_reviews.drop_duplicates()
glassdoor_reviews = glassdoor_reviews.loc[glassdoor_reviews['employer_name'] != 't']
glassdoor_reviews = glassdoor_reviews.loc[glassdoor_reviews['employer_name'] != 'c']

In [13]:
l = glassdoor_reviews[['employer_name', 'employer_id']].drop_duplicates()
l.to_csv(os.path.join(data_dir, 'companies_only.csv'))

### Processing Forbes Data

In [17]:
forbes_df = pd.read_csv(os.path.join(data_dir, "forbes_most_admired_2021.csv"), names=['val'])
wide_data = []
long_data = forbes_df['val'].tolist()
for i in range(0, len(long_data)):
    if i % 2 == 1:
        continue
    row = [int(long_data[i]), long_data[i+1]]
    wide_data.append(row)
forbes_df = pd.DataFrame(wide_data, columns = ["rank", "company"])

In [18]:
forbes_df['key']=forbes_df.company.progress_apply(lambda x : [process.extract(x, glassdoor_reviews.employer_name, limit=1, processor=False, scorer = fuzz.WRatio)][0][0][0])

100%|██████████| 332/332 [06:53<00:00,  1.24s/it]


In [19]:
forbes_df = forbes_df.merge(glassdoor_reviews,left_on='key',right_on='employer_name')
forbes_df = forbes_df[['rank', 'company', 'employer_id', 'employer_name']]
forbes_df.to_csv(os.path.join(data_dir, 'forbes_glassdoor_2019.csv'), index=False)

### Processing Harris Poll Data

In [168]:
harris_df = pd.read_csv(os.path.join(data_dir, "harris_reputation_poll.csv"))
harris_df.columns = ['rank', 'company', 'score']
harris_df['rank'] = harris_df['rank'].apply(lambda x : x[1:])

In [169]:
harris_df['key']=harris_df.company.progress_apply(lambda x : [process.extract(x, glassdoor_reviews.employer_name, limit=1, processor=False, scorer = fuzz.WRatio)][0][0][0])


100%|██████████| 100/100 [02:01<00:00,  1.21s/it]


In [170]:
harris_df = harris_df.merge(glassdoor_reviews,left_on='key',right_on='employer_name')
harris_df = harris_df[['rank', 'company', 'employer_id', 'employer_name', 'score']]
harris_df = harris_df.drop_duplicates()

In [171]:
harris_df

,rank,company,employer_id,employer_name,score
0,1,Patagonia,5474,"Patagonia, Inc.",82.7
1,2,Honda Motor Company,7474,Hyundai Motor Company,81.6
2,3,Moderna,453959,"Moderna, Inc.",81.3
3,4,Chick-fil-A,5873,"Chick-fil-A, Inc.",81.1
4,5,SpaceX,693210,zSpace,81.1
...,...,...,...,...,...
98,96,Sears Holdings Corporation,37984,Sears Holdings Corporation,61.2
99,97,Wish.com,686514,Wish,60.7
100,98,Facebook,40772,"Facebook, Inc.",60.0
101,99,Fox Corporation,2938629,Fox Corporation,59.2


In [172]:
harris_df.to_csv(os.path.join(data_dir, 'harris_glassdoor_2019.csv'), index=False)

### Postscript

The matched datasetes are then visually inspected for quality. Any matches that are wrong are manually corrected by comparing names in 'companies_only.csv' to names in the two reputation datasets. The name corrections are saved into the original files.

### Processing Forbes 2019

In [66]:
forbes_2021_df = pd.read_csv(os.path.join(data_dir, "forbes_glassdoor_2021.csv"))
forbes_df = pd.read_csv(os.path.join(data_dir, "forbes_most_admired_2019.csv"), names=['val'])
wide_data = []
long_data = forbes_df['val'].tolist()
for i in range(0, len(long_data)):
    if i % 2 == 1:
        continue
    row = [int(long_data[i].replace(',', '')), long_data[i+1]]
    wide_data.append(row)
forbes_df = pd.DataFrame(wide_data, columns = ["rank", "company"])

In [67]:
forbes_df=forbes_df.merge(forbes_2021_df[['company', 'employer_name', 'employer_id']],left_on='company',right_on='company', how='left')


In [68]:
import numpy as np
forbes_df['employer_name']=forbes_df.progress_apply(lambda x : [process.extract(x['company'], glassdoor_reviews.employer_name, limit=1, processor=False, scorer = fuzz.WRatio)][0][0][0] if pd.isna(x['employer_name']) else x['employer_name'], axis=1)


100%|██████████| 337/337 [01:41<00:00,  3.32it/s]


In [78]:
combined_df = forbes_df[['company', 'employer_name']].drop_duplicates().merge(glassdoor_reviews, on='employer_name', how='left')

In [79]:
combined_df

,company,employer_name,employer_id
0,Apple,Apple Inc.,1138
1,Amazon,"Amazon.com, Inc.",6036
2,Berkshire Hathaway,Berkshire Hathaway Inc.,94
3,Walt Disney,The Walt Disney Company,717
4,Starbucks,Starbucks Corporation,2202
...,...,...,...
333,Xcel Energy,Xcel Energy Inc.,487
334,XPO Logistics,XPO Logistics,3221547
335,Yum Brands,"YUM! Brands, Inc.",6995
336,Yum China Holdings,Laboratory Corporation of America Holdings,1679


In [80]:
combined_df.to_csv(os.path.join(data_dir, 'forbes_glassdoor_2019.csv'), index=False)